# **Step 5 - A tool to get the current date**

I'm sure you got the solution: **Add another tool that returns the current date time.** 😉

So, let's do it.

The same old code you've seen before for initialzing the client & data provider. Just run it.

In [ ]:
# Imports
import os
import json
import datetime
import pytz
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime
from services.data_provider import DataProvider
from termcolor import cprint

# Environment setup
load_dotenv()
api_key=os.environ['AZURE_OPENAI_API_KEY']
api_version=os.environ['AZURE_OPENAI_API_VERSION']
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
local_timezone = os.environ['TIMEZONE']

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=api_key,  
  api_version = api_version
  )

# This the data access layer
db:DataProvider=DataProvider()

messages = []

## Create a new function to return the current date & time 

In [ ]:
# Function to get the current date and time in the format <day-name>, dd/mm/yyyy hh:mm
def get_current_datetime():       
    local_tz = pytz.timezone(local_timezone)
    return json.dumps({
        "current_datetime": datetime.now(local_tz).strftime("%A, %d/%m/%Y %H:%M")
    })


## Inform the LLM of the new function via the system message 
The `system_message` has been slightly modifed to enforce the use of the new `get_current_datetime`, sometimes it's necessary to be more specific regarding how to use the tools.

In [ ]:
# Updated system message, we now hint the LLM to use the what returned from get_current_datetime as reference date for all date calculations
system_message = f'''
Role: An assistant with expertise in handling questions about operative schedule and available menu of a food delivery service.
 
Instructions:
- Provide information about the available menu and the opening schedule.
- Kindly deny any request not regarding opening schedule and available menu.
- Always assume the date and time returned by the get_current_datetime tool as the reference date for all date calculations.
- If a date is not indicated use the reference date.
- If a time is not indicated assume it is 12 PM.
- Assume the following format: 'DD/MM/YYYY HH:MM' as the standard format for dates.

Steps:
  - Opening Schedule:
      1. Always use the calculated date based on reference date to check and return the opening schedule.
      2. Always indicate the desired data and time including weekday in the response.
      
  - Available menu:
      1. When the user asks for the a menu you should respond using the following template enclosed in triple quotes:

      ```
      Menu for <day-name> <requested date>
      
      Pasta
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..      
      
      
      Pizza
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      
      Today's Special Pizzas
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. ..   
      
      Drinks
      
      1. <name> - <price> 
      2. <name> - <price> 
      3. ...
      4. .. 
      
      Dessert
      
      1. <name> - <price> - <ingredients> - <label>
      2. <name> - <price> - <ingredients> - <label>
      3. ...
      4. .. 
      ```  
      
      2. If the request is about specific menu entries (e.g. 'do you have pasta', 'do you offer vegetarian food' or 'are there any specials?' ), together with the menu kindle answer the question.     
      3. If no specials are available for the day, do not include that section into menu.
      4. If no menu is available, kindly reply that there you can serve any food for that day..
      5. If the request relates to an item that is not available in the menu, kindly reply that it cannot be ordered.      
  
Expectation:
  - Provide a seamless experience to the user, by providing the requested information in a kind and timely manner, including all the necessary details and guiding the user to a possible follow up step.

Narrowing:  
  1. Deny all the requests referring to a date antecedent the reference date.
'''

## Describe the functions in the 'tools' for use when calling the LLM


Same code as before with the addition of a new tool (or function, depending on what term you like most) `get_current_datetime`.   First the functions: 

In [ ]:

# Returns the operative schedule of the pizzeria on provided date (Monday closed, 9 am to 9 pm other days) for the user_id provided (if indicated)
def operative_schedule(date:str, user_id:str=None):
    weekday= datetime.strptime(date, "%d/%m/%Y %H:%M").strftime("%A")    
        
    # Check if the user is a special client
    is_special = db.is_special_client(user_id)
    # Get the operative schedule from the database
    schedule=db.get_schedule(day=weekday, is_special=is_special) 
    # Check if the requested date/time is within the working schedule       
    if schedule.status == "open":        
        time= datetime.strptime(date, "%d/%m/%Y %H:%M").time()
        opening = datetime.strptime(schedule.start, "%H:%M").time()
        closing = datetime.strptime(schedule.end, "%H:%M").time()        
        if time < opening or time > closing:
            schedule.status = "closed"            
        
    return schedule.model_dump_json()
# endregion
    
# Returns the menu on the provided date    
def get_menu(date:str):
    target_date= datetime.strptime(date, "%d/%m/%Y %H:%M") 
    weekday= target_date.strftime("%A")
    
    # Get the menu from the database
    menu= db.get_menu(day=weekday)
    #return the response as JSON string    
    return menu.model_dump_json()

# Function to get the current date and time in the format <day-name>, dd/mm/yyyy hh:mm
def get_current_datetime():       
    local_tz = pytz.timezone(local_timezone)
    return json.dumps({
        "current_datetime": datetime.now(local_tz).strftime("%A, %d/%m/%Y %H:%M")
    })

# We create a dictionary with the available functions, using function name as key, note the new added functions "get_current_datetime"
available_functions = {
            "operative_schedule": operative_schedule,
            "get_menu": get_menu,
            "get_current_datetime": get_current_datetime
    }

Again, the functions are described in the format defined by OpenAI: 

https://platform.openai.com/docs/guides/function-calling

The definition of `get_current_datetime` has been added, notice the absence of parameters in this case.


In [ ]:
# Define the function available to the LLM, note the new added function "get_current_datetime"
tools = [
        {        
            "type": "function",    
            "function": {
                "name": "operative_schedule",
                "description": "Useful when you need to know the operative schedule on a specific date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date for which you want to know the schedule in format 'DD/MM/YYYY HH:MM' (e.g. '31/12/2023 15:00').",
                        },
                        "user_id": {
                            "type": "string",
                            "description": "The optional user id in form '#<4-digits-number>'.",
                        }                        
                    },
                    "required": ["date"]
                }
            },            
        },
               {        
            "type": "function",    
            "function": {
                "name": "get_menu",
                "description": "Useful when you need to know the menu for a specific date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date for which you want to know the available menu in format 'DD/MM/YYYY HH:MM' (e.g. '29/05/2024 17:00').",
                        }                  
                    },
                    "required": ["date"]
                }
            },            
        },
        {        
            "type": "function",    
            "function": {
                "name": "get_current_datetime",
                "description": "Useful when you need to know the current date and time for date and time based operations.",
                "parameters": {}
            },            
        }
    ]

## Invoke the LLM with the tools

The `invoke_llm()` function includes the `tools` (above) in the request.  


In [ ]:
# Invoke the LLM with the provided input, if none provided it just uses the existing messages
def invoke_llm(input:str=None)->str:
    if input:
          messages.append({'role': 'user', 'content': input})
    response = client.chat.completions.create(
    model=deployment,    
    messages = messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.3)

    return response.choices[0].message

And here is the same code as before, notice that we no longer provide the current datetime together with the `user_message`.

Change the `user message` below and play with the next cell to see the different behavior depending on how the `user_message` look like.

In [ ]:


# This is the user request, play with it and re-run this cell to see how the system responds
user_message = "what pizza can i order next Tuesday? are you open on that day?"


messages = [{'role': 'system', 'content': system_message}]

# Here's the new input request, note that now we are no longer providing the current date and time.
input_request = f"Handle the following user request enclosed in triple backticks: ```{user_message}```"
cprint(f"Input: {input_request}", "blue") 

while True:
    # We invoke the LLM with the input request
    response = invoke_llm(input=input_request)    
    # We check if the response contains tool calls
    if response.tool_calls:
        cprint(response, "cyan")
        # Note that we add the request from the LLM to the messages list
        messages.append(response)
        for tool_call in response.tool_calls:
            # We get the function to call from the available_functions dictionary
            function_to_call = available_functions[tool_call.function.name]
            # We extract the arguments from the tool call            
            function_args = json.loads(tool_call.function.arguments)
            # We call the function with the extracted arguments
            cprint (f"Invoking function: {tool_call.function.name}...", "green")
            function_response = function_to_call(**function_args)
            cprint (f"Function call completed...", "yellow")
            # We append the function response to the messages list, note that we add the tool_call id to the response to let the LLM identify it
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool", 
                    "name": tool_call.function.name,
                    "content": function_response,
                }) 
        # We set input request to None to let the LLM know that it should continue with the existing conversation that contains the function responses
        input_request = None
        cprint (f"Control back to the LLM...", "magenta")
    else:
        # If the response does not contain tool calls, it means LLM has completed the required tasks
        cprint (response.content, "yellow")
        break    

## Why use this tool at all?  

As you can see, the LLM now takes care of using the `get_current_datetime` tool to get the current date.  

**Yes, but...**  

Some of you might argue that adding a tool for getting the current datetime doesn't make a lot of sense compared to adding it to the `user_message` as before, and for this specific example that is correct. 

We are simply using it as an example of doing processing only when necessary.  For example, it could also be that the date is not required to handle the user's request. 

*Example*:  

Run the cell above with `user_message` = *"How are you today?"*  
Did you see that none of the tools have been called?


## Congratulations, you've created your first agent 


**What exactly is an 'Agent'?**  

An agent is an advanced AI system designed for completing tasks that need sequential reasoning. They can plan, remember past conversations, and use different tools to adjust their responses based on the request. 

And is exactly what you've built just now. 🙂

<img src="../docs/images/agent-diagram.png" width="800">

**Can we make it simpler?**  

You might already been noticed that what we've built right now is:  

1. **Error prone:** Editing the Tool object might easily lead to mistakes.
2. The code that handles the orchestration could be easily made **generic**, in the end there's nothing specific to our Food ordering agent there.

And the good new is that in real-world, few people create agents this way, preferring instead to use frameworks that abstract all these (important) details and let you concentrate on the important part that are: `system_message` and `tools`

Several frameworks you may have heard of include:  

- [Langchain](https://www.langchain.com/)
- [Semantic Kernel](https://github.com/microsoft/semantic-kernel)
- [LLamaIndex](https://www.llamaindex.ai/)
- [Haystack](https://haystack.deepset.ai/)


In [Notebook 06](06_use_langchain.ipynb) we are going to refactor our code to use Langchain.

